In [1]:
import pickle
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import sklearn.manifold
import numpy as np
import pandas as pd
import re
import multiprocessing
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt 
import seaborn as sns
import time
import collections

In [22]:
file = open("weat_oc.txt", "r" ) 
file=file.read()

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/anganab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import nltk 
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(file.lower())

In [5]:
raw_sentences

["religious beliefs and figures: 'god', 'gods', 'bible', 'mankind', 'faith', 'christ', 'spiritual', 'christian', 'religion', 'jesus'.",
 "family relationships: 'mum', 'mother', 'mom', 'mums', 'parent', 'family', 'parents', 'baby', 'dad', 'father'.",
 "male_terms: male, man, boy, brother, he, him, his, son, 'manaia', 'tane', 'kai', 'ariki', 'mika', 'koa', 'rangi', 'kane', 'tama', 'hemi'.",
 "female_terms: sister, mother, aunt, grandmother, daughter, she, hers, her, 'aroha', 'moana', 'tui', 'lani', 'kahurangi', 'ariana', 'malie', 'marama', 'ava', 'kaia'.",
 "music record and artists: 'music', 'album', 'albums', 'jazz', 'songs', 'hits', 'musicians', 'artists', 'recordings', 'blues'.",
 "woordworking plans and projects: 'plans', 'furniture', 'woodwork', 'wood', 'woodcraft', 'woodworking', 'plywood', 'carpentry', 'cabinets', 'wooden'.",
 "male_terms: male, man, boy, brother, he, him, his, son, 'manaia', 'tane', 'kai', 'ariki', 'mika', 'koa', 'rangi', 'kane', 'tama', 'hemi'.",
 "female_terms

In [21]:
import pandas as pd 

# df1 = pd.read_csv("GeoWAC_NA.csv")
df1 = pd.read_csv("un_oc.csv")

# df2 = pd.read_csv("Female_aligned_asia_itr2_52pairs_30context.csv")

In [ ]:
df1.fillna("", inplace=True) 
df1['st'] = df1['post'] + df1['title']

In [7]:
df1.shape

(233, 5)

In [86]:
df1 = df1[:500]

In [8]:
file_ = list(df1.text.values)

In [9]:
file2 = ''.join(map(str, file_))

In [10]:
file2[0]

'\ufeff'

In [11]:
#Split the corpus into sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences2 = tokenizer.tokenize(file2)

In [12]:
print("Corpus is now {0} characters long".format(len(file2)))

Corpus is now 3834740 characters long


In [13]:
#convert into a list of words
#remove unnecessary, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean=re.sub('[\n]',' ',raw.lower())
    clean = re.sub("[^a-zA-Z]"," ", raw.lower())
    words = clean.split()
    #words = [ps.stem(w) for w in words if not w in stop_words]  
    words = [w for w in words if not w in stop_words] 
    return words

In [14]:
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences2:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))
#print(sentences[1])

In [15]:
token_count = sum([len(sentence) for sentence in sentences])
print("The corpus contains {0:,} tokens".format(token_count))

The corpus contains 326,643 tokens


In [16]:
def getWordList(sentences,minCount):
    wordList = []
    count = Counter()
    for sentence in sentences:
        for word in sentence:
            count[word] += 1
    for word,c in count.items():
        if c >= minCount:
            wordList.append(word)
    return wordList

In [17]:
WordList=getWordList(sentences,minCount=5) # for min frequency of a word which should be 5
print(WordList)

['mr', 'president', 'steps', 'lead', 'podium', 'speaking', 'united', 'nations', 'general', 'assembly', 'always', 'great', 'honour', 'privilege', 'today', 'take', 'added', 'significance', 'delegation', 'pleasure', 'conveying', 'government', 'people', 'federal', 'republic', 'nigeria', 'congratulations', 'best', 'wishes', 'unanimous', 'election', 'forty', 'fourth', 'session', 'furthermore', 'personal', 'message', 'father', 'prime', 'minister', 'vanuatu', 'regards', 'friend', 'man', 'knows', 'proud', 'son', 'also', 'recall', 'title', 'chief', 'upon', 'ceremony', 'visit', 'made', 'capacity', 'chairman', 'special', 'committee', 'apartheid', 'thus', 'pride', 'shared', 'resist', 'sake', 'life', 'repeating', 'many', 'words', 'already', 'spoken', 'praise', 'diplomatic', 'experience', 'skills', 'instead', 'merely', 'associate', 'sentiments', 'express', 'prefer', 'draw', 'attention', 'four', 'us', 'privileged', 'know', 'personally', 'find', 'appropriate', 'describing', 'sincerity', 'courage', 'dig

In [18]:
from gensim.models import FastText


In [19]:
model = FastText(sentences,vector_size=300,window=5,min_count=5,seed=132,workers=multiprocessing.cpu_count(),batch_words=1)

In [30]:
model2 = Word2Vec(sentences,size=300,window=5,min_count=5,seed=42,workers=multiprocessing.cpu_count(),batch_words=1)

In [24]:
def Stability2(word,similar1,similar2):    
    overlap=0
    for i in range(len(similar1)):
        for j in range(len(similar2)):
            if similar1[i][0] == similar2[j][0]:
                overlap=overlap+1
    return overlap

In [41]:
df['Stability(%) IU'] = pd.to_numeric(df['Stability(%) IU'])
s = df['Stability(%) IU'].sum()
print(s)

4959061.780428033


In [26]:
def target_attribute(target_one,target_two, target_one_words, attribute_one,attribute_two, attribute_one_words, target_two_words, attribute_two_words):
    cos=[]
    s=0
    s1=[]
    s2=[]
    S=[]
    n=0
    for i in range(0, len(target_one_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_one_words[i][:-1]
                at1 = attribute_one_words[k][:-1]
                try:
                    cos1= model.wv.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_one_words[i][:-1]
                at2 = attribute_two_words[k][:-1]
                try:
                    cos2= model.wv.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s1.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
            n=n+1
            # print(S)
    for i in range(0, len(target_two_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_two_words[i][:-1]
                at1 = attribute_one_words[k][:-1]
                try:
                    cos1= model.wv.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_two_words[i][:-1]
                at2 = attribute_two_words[k][:-1]
                try:
                    cos2= model.wv.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s2.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
            # print(S)
    s=np.sum(s1)-np.sum(s2)
    stdev=np.std(S)
    # print(s)
    print(target_one + ' vs ' + target_two  + ' , ' +attribute_one + ' vs ' + attribute_two +', d = ' + str(s/(stdev*n)))
    #print(str(s/(stdev*n)))

In [ ]:
for i in range(0, len(raw_sentences), 4):
    print(i)
    words=raw_sentences[i].split(":")
    # print(words)
    target_one = words[0]
    # print(target_one)
    target_one_words = list(words[1].split(','))
    target_one_words = [elem.strip().replace(" ", "").replace("'", "") for elem in target_one_words]  
    target_one_words = [elem.strip().replace(".","") for elem in target_one_words]  


    words1=raw_sentences[i+1].split(":")
    # print(words)
    target_two = words1[0]
    # print(target_one)
    target_two_words = list(words1[1].split(','))
    target_two_words = [elem.strip().replace(" ", "").replace("'", "") for elem in target_two_words]  
    target_two_words = [elem.strip().replace(".","") for elem in target_two_words]  

    
    words1=raw_sentences[i+2].split(":")
    # print(words)
    attribute_one = words1[0]
    # print(target_one)
    attribute_one_words = list(words1[1].split(','))
    attribute_one_words = [elem.strip().replace(" ", "").replace("'", "") for elem in attribute_one_words]  
    attribute_one_words = [elem.strip().replace(".","") for elem in attribute_one_words]  

    
    words1=raw_sentences[i+3].split(":")
    # print(words)
    attribute_two = words1[0]
    # print(target_one)
    attribute_two_words = list(words1[1].split(','))
    attribute_two_words = [elem.strip().replace(" ", "").replace("'", "") for elem in attribute_two_words]  
    attribute_two_words = [elem.strip().replace(".","") for elem in attribute_two_words]  

    
    # i = i+4
    # print(target_one_words)
    # words1=raw_sentences[i*4+1].split()
    # target_two = words1[0][:-1]
    # target_two_words = words1[1:]
    # words2=raw_sentences[i*4+2].split()
    # attribute_one = words2[0][:-1]
    # attribute_one_words = words2[1:]
    # words3 = raw_sentences[i*4+3].split()
    # attribute_two = words3[0][:-1]
    # attribute_two_words = words3[1:]
    
    print(target_attribute(target_one, target_two, target_one_words, attribute_one,attribute_two, attribute_one_words, target_two_words, attribute_two_words))